In [1]:
import torch
import deepsensor
import deepsensor.torch
from deepsensor.train import set_gpu_default_device
set_gpu_default_device()

In [2]:
from deepsensor.train import Trainer
from deepsensor.model import ConvNP
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from deepsensor.data import DataProcessor, TaskLoader
from tqdm import tqdm
from scipy.special import sph_harm

# Load first three files from dataset
data_path = '/nfs/turbo/seas-dannes/urop-2024-bias/cfs-forecasts/cfs_2012-2023.zarr'
ds0 = xr.open_zarr(data_path, consolidated=True)

# Check the contents of the dataset
ds = ds0.isel(lead=0, drop=True)
ds = ds.sortby("time")
#ds = ds.sel(time=ds.time.dt.hour == 0)

#ds = ds.sel(time=slice("2012-01-01", "2015-12-31"))  # Select only the first three files




['2012-01-01T00:00:00.000000000' '2012-01-01T06:00:00.000000000'
 '2012-01-01T12:00:00.000000000' ... '2024-01-01T06:00:00.000000000'
 '2024-01-01T12:00:00.000000000' '2024-01-01T18:00:00.000000000']


['2012-01-01T00:00:00.000000000' '2012-01-01T06:00:00.000000000'
 '2012-01-01T12:00:00.000000000' ... '2015-01-01T06:00:00.000000000'
 '2015-01-01T12:00:00.000000000' '2015-01-01T18:00:00.000000000']


<xarray.Dataset> Size: 3GB
Dimensions:            (time: 4206, latitude: 181, longitude: 360)
Coordinates:
  * latitude           (latitude) float64 1kB -90.0 -89.0 -88.0 ... 89.0 90.0
  * longitude          (longitude) float64 3kB 0.0 1.0 2.0 ... 357.0 358.0 359.0
  * time               (time) datetime64[ns] 34kB 2012-01-01 ... 2015-01-01T1...
Data variables:
    LHTFL_surface      (time, latitude, longitude) float32 1GB dask.array<chunksize=(1, 91, 180), meta=np.ndarray>
    SHTFL_surface      (time, latitude, longitude) float32 1GB dask.array<chunksize=(1, 91, 180), meta=np.ndarray>
    TMP_2maboveground  (time, latitude, longitude) float32 1GB dask.array<chunksize=(1, 91, 180), meta=np.ndarray>


In [7]:
# Apply bias correction function

def periodic_step_bias(data, period=5, step=3):
    time_factor = ((np.arange(data.shape[0]) // period) % 2).reshape(-1, 1, 1)
    return data + step * time_factor



In [8]:
print(ds.sel(time=slice("2012-01-01T00:00:00.000000000", "2012-01-03T00:00:00.000000000")))

<xarray.Dataset> Size: 7MB
Dimensions:            (time: 9, latitude: 181, longitude: 360)
Coordinates:
  * latitude           (latitude) float64 1kB -90.0 -89.0 -88.0 ... 89.0 90.0
  * longitude          (longitude) float64 3kB 0.0 1.0 2.0 ... 357.0 358.0 359.0
  * time               (time) datetime64[ns] 72B 2012-01-01 ... 2012-01-03
Data variables:
    LHTFL_surface      (time, latitude, longitude) float32 2MB dask.array<chunksize=(1, 91, 180), meta=np.ndarray>
    SHTFL_surface      (time, latitude, longitude) float32 2MB dask.array<chunksize=(1, 91, 180), meta=np.ndarray>
    TMP_2maboveground  (time, latitude, longitude) float32 2MB dask.array<chunksize=(1, 91, 180), meta=np.ndarray>


In [9]:
data_processor = DataProcessor(x1_name="latitude", x2_name="longitude")
_ = data_processor(ds.sel(time=slice("2012-01-01T00:00:00.000000000", "2012-01-31T00:00:00.000000000")))
ds_processed = data_processor(ds)
print(data_processor)

DataProcessor with normalisation params:
{'LHTFL_surface': {'method': 'mean_std',
                   'params': {'mean': 66.60301208496094,
                              'std': 65.60218048095703}},
 'SHTFL_surface': {'method': 'mean_std',
                   'params': {'mean': 6.118175506591797,
                              'std': 31.839258193969727}},
 'TMP_2maboveground': {'method': 'mean_std',
                       'params': {'mean': 276.7368469238281,
                                  'std': 20.317398071289062}},
 'coords': {'time': {'name': 'time'},
            'x1': {'map': (-90.0, 269.0), 'name': 'latitude'},
            'x2': {'map': (0.0, 359.0), 'name': 'longitude'}}}


In [10]:
print(ds_processed)

<xarray.Dataset> Size: 3GB
Dimensions:            (time: 4206, x1: 181, x2: 360)
Coordinates:
  * time               (time) datetime64[ns] 34kB 2012-01-01 ... 2015-01-01T1...
  * x1                 (x1) float64 1kB 0.0 0.002786 0.005571 ... 0.4986 0.5014
  * x2                 (x2) float64 3kB 0.0 0.002786 0.005571 ... 0.9972 1.0
Data variables:
    LHTFL_surface      (time, x1, x2) float32 1GB dask.array<chunksize=(1, 91, 180), meta=np.ndarray>
    SHTFL_surface      (time, x1, x2) float32 1GB dask.array<chunksize=(1, 91, 180), meta=np.ndarray>
    TMP_2maboveground  (time, x1, x2) float32 1GB dask.array<chunksize=(1, 91, 180), meta=np.ndarray>


In [11]:
train_range = ("2012-01-01", "2014-12-31")
val_range = ("2015-01-01", "2015-12-31")

In [12]:
def add_bias_function(data, bias_function, **kwargs):
    return data + bias_function(data, **kwargs)

# Define TaskLoader
biased_contexts = [
    #add_bias_function(ds_processed["TMP_2maboveground"], spherical_harmonic_bias),
    #add_bias_function(ds_processed["TMP_2maboveground"], random_noise_bias),
    #add_bias_function(ds_processed["TMP_2maboveground"], linear_trend_bias),
    add_bias_function(ds_processed["TMP_2maboveground"], periodic_step_bias),
]

task_loader = TaskLoader(
    context=biased_contexts,  
    target=ds_processed["TMP_2maboveground"],
)

print(task_loader)

TaskLoader(1 context sets, 1 target sets)
Context variable IDs: (('TMP_2maboveground',),)
Target variable IDs: (('TMP_2maboveground',),)


In [13]:
import time
import sys
def gen_tasks(dates, progress=True):
    tasks = []
    for date in tqdm(dates):
        date = np.datetime64(date)  # Ensure consistent datetime format
        try:
            task = task_loader(date, context_sampling="all", target_sampling="all")  
            tasks.append(task)
        except KeyError:
            print(f"Skipping date {date} as it is not found in dataset.")
        sys.stdout.flush()
    print(f"Finished generating {len(tasks)} tasks.")
    return tasks


# Define the ConvNP model
model = ConvNP(data_processor, task_loader, dim_yc=(1,))

# Train the model
trainer = Trainer(model)

dim_yt inferred from TaskLoader: 1
dim_aux_t inferred from TaskLoader: 0
internal_density inferred from TaskLoader: 359
encoder_scales inferred from TaskLoader: [np.float32(0.0013927576)]
decoder_scale inferred from TaskLoader: 0.002785515320334262


In [14]:
ds["time"] = pd.to_datetime(ds["time"].values).strftime("%Y-%m-%d %H:%M:%S")

In [15]:
print(ds.time)

<xarray.DataArray 'time' (time: 4206)> Size: 34kB
array(['2012-01-01 00:00:00', '2012-01-01 06:00:00', '2012-01-01 12:00:00',
       ..., '2015-01-01 06:00:00', '2015-01-01 12:00:00',
       '2015-01-01 18:00:00'], dtype=object)
Coordinates:
  * time     (time) object 34kB '2012-01-01 00:00:00' ... '2015-01-01 18:00:00'


In [16]:
losses = []
val_rmses = []
train_rmses = []

In [17]:
def compute_val_rmse(model, val_tasks):
    errors = []
    target_var_ID = task_loader.target_var_IDs[0][0]  # assume 1st target set and 1D
    for task in val_tasks:
        mean = data_processor.map_array(model.mean(task), target_var_ID, unnorm=True)
        true = data_processor.map_array(task["Y_t"][0], target_var_ID, unnorm=True)
        errors.extend(np.abs(mean - true))
    return np.sqrt(np.mean(np.concatenate(errors) ** 2))


In [18]:
def compute_train_rmse(model, train_tasks):
    errors = []
    context_var_ID = task_loader.context_var_IDs[0][0]  # assume 1st target set and 1D
    for task in train_tasks:
        mean = data_processor.map_array(model.mean(task), context_var_ID, unnorm=True)
        true = data_processor.map_array(task["Y_t"][0], context_var_ID, unnorm=True)
        errors.extend(np.abs(mean - true))
    return np.sqrt(np.mean(np.concatenate(errors) ** 2))


In [19]:
val_dates = pd.date_range(val_range[0], val_range[1])
val_tasks = gen_tasks(val_dates)
_ = model(val_tasks[0])
print(f"Model has {deepsensor.backend.nps.num_params(model.model):,} parameters")

  0%|          | 1/365 [00:00<02:39,  2.28it/s]

Skipping date 2015-01-02T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-01-03T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-01-04T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-01-05T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-01-06T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-01-07T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-01-08T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-01-09T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-01-10T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-01-11T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-01-12T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-01-13T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-01-14T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-01-15T00:00:00.000000 as it is not found in d

 53%|█████▎    | 195/365 [00:00<00:00, 478.59it/s]

Skipping date 2015-07-15T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-07-16T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-07-17T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-07-18T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-07-19T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-07-20T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-07-21T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-07-22T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-07-23T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-07-24T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-07-25T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-07-26T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-07-27T00:00:00.000000 as it is not found in dataset.
Skipping date 2015-07-28T00:00:00.000000 as it is not found in d

100%|██████████| 365/365 [00:00<00:00, 584.37it/s]


Finished generating 1 tasks.


/home/whruiray/deepsensor_gpu_2/lib/python3.10/site-packages/lab/types.py:178: DeprecationWarning: numpy.core is deprecated and has been renamed to numpy._core. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numerictypes.
  for name in np.core.numerictypes.__all__ + ["bool"]:


Model has 1,027,844 parameters


In [20]:
# Train model
#deepsensor_folder = "/home/whruiray/deepsensor_config/"


val_rmse_best = np.inf
train_rmse_best = np.inf


trainer = Trainer(model, lr=5e-5)
train_tasks = gen_tasks(pd.date_range(train_range[0], train_range[1]), progress=False)

  5%|▌         | 60/1096 [00:23<06:52,  2.51it/s]

Skipping date 2012-03-01T00:00:00.000000 as it is not found in dataset.


 16%|█▌        | 176/1096 [01:10<05:56,  2.58it/s]

Skipping date 2012-06-25T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-06-26T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-06-27T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-06-28T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-06-29T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-06-30T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-07-01T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-07-02T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-07-03T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-07-04T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-07-05T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-07-06T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-07-07T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-07-08T00:00:00.000000 as it is not found in d

 20%|█▉        | 215/1096 [01:10<00:27, 31.58it/s]

...

Skipping date 2013-06-17T00:00:00.000000 as it is not found in dataset.
Skipping date 2013-06-18T00:00:00.000000 as it is not found in dataset.


 98%|█████████▊| 1069/1096 [06:36<00:10,  2.60it/s]

Skipping date 2014-12-05T00:00:00.000000 as it is not found in dataset.
Skipping date 2014-12-06T00:00:00.000000 as it is not found in dataset.


100%|██████████| 1096/1096 [06:46<00:00,  2.70it/s]

Finished generating 1049 tasks.


In [ ]:
for epoch in tqdm(range(10)):
    #train_tasks = gen_tasks(pd.date_range(train_range[0], train_range[1]), progress=False)
    
    print(f"Training batch for Epoch {epoch+1} started...")
    sys.stdout.flush()
    
    batch_losses = trainer(train_tasks)
    
    
    losses.append(np.mean(batch_losses))
    print(f"Training batch for Epoch {epoch+1} complete...")
    sys.stdout.flush()
   
    train_rmses.append(compute_train_rmse(model, train_tasks))
    print(f"Epoch {epoch+1} - Loss: {losses[-1]:.4f}, Validation RMSE: {train_rmses[-1]:.4f}")
    print(f"Epoch {epoch+1} completed in {time.time()} seconds.")
    sys.stdout.flush()
    if train_rmses[-1] < train_rmse_best:
        train_rmse_best = train_rmses[-1]

    val_rmses.append(compute_val_rmse(model, val_tasks))
    print(f"Epoch {epoch+1} - Loss: {losses[-1]:.4f}, Validation RMSE: {val_rmses[-1]:.4f}")
    print(f"Epoch {epoch+1} completed in {time.time()} seconds.")
    sys.stdout.flush()
    if val_rmses[-1] < val_rmse_best:
        val_rmse_best = val_rmses[-1]

        #model.save(deepsensor_folder)

  0%|          | 0/10 [00:00<?, ?it/s]

Training batch for Epoch 1 started...
Training batch for Epoch 1 complete...
Epoch 1 - Loss: -0.8949, Validation RMSE: 1.8689
Epoch 1 completed in 1744040768.6207387 seconds.
Epoch 1 - Loss: -0.8949, Validation RMSE: 1.9150
Epoch 1 completed in 1744040768.7215345 seconds.


 10%|█         | 1/10 [06:06<54:58, 366.52s/it]

Training batch for Epoch 2 started...
Training batch for Epoch 2 complete...
Epoch 2 - Loss: -1.1087, Validation RMSE: 1.7299
Epoch 2 completed in 1744041135.9032671 seconds.
Epoch 2 - Loss: -1.1087, Validation RMSE: 1.7196
Epoch 2 completed in 1744041136.005336 seconds.


 20%|██        | 2/10 [12:13<48:55, 366.97s/it]

Training batch for Epoch 3 started...
Training batch for Epoch 3 complete...
Epoch 3 - Loss: -1.1684, Validation RMSE: 1.7154
Epoch 3 completed in 1744041503.7912586 seconds.
Epoch 3 - Loss: -1.1684, Validation RMSE: 1.7345
Epoch 3 completed in 1744041503.893594 seconds.


 30%|███       | 3/10 [18:21<42:51, 367.39s/it]

Training batch for Epoch 4 started...
Training batch for Epoch 4 complete...
Epoch 4 - Loss: -1.2064, Validation RMSE: 1.7263
Epoch 4 completed in 1744041871.8972657 seconds.
Epoch 4 - Loss: -1.2064, Validation RMSE: 1.7282
Epoch 4 completed in 1744041871.9996293 seconds.


 40%|████      | 4/10 [24:29<36:46, 367.67s/it]

Training batch for Epoch 5 started...


In [ ]:
import os
import matplotlib.pyplot as plt

# Plot training metrics with adjusted scales
fig, ax1 = plt.subplots(figsize=(10, 6))

# Smooth the loss values (using simple moving average)
window_size = 5
smoothed_losses = np.convolve(losses, np.ones(window_size)/window_size, mode='valid')

# Plot training loss
ax1.plot(range(len(smoothed_losses)), smoothed_losses, 'b-', label='Training Loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss', color='b')
ax1.tick_params('y', colors='b')

# Create a second y-axis for validation RMSE
ax2 = ax1.twinx()
ax2.plot(range(len(val_rmses)), val_rmses, 'r-', label='Validation RMSE')
ax2.set_ylabel('RMSE', color='r')
ax2.tick_params('y', colors='r')

# Adjust scales to show downward trend
if len(losses) > 0:
    loss_min, loss_max = min(smoothed_losses), max(smoothed_losses)
    ax1.set_ylim(loss_min - 0.1*(loss_max-loss_min), loss_max + 0.1*(loss_max-loss_min))

if len(val_rmses) > 0:
    rmse_min, rmse_max = min(val_rmses), max(val_rmses)
    ax2.set_ylim(rmse_min - 0.1*(rmse_max-rmse_min), rmse_max + 0.1*(rmse_max-rmse_min))

plt.title('Training Loss and Validation RMSE')
fig.tight_layout()
plt.show()


In [ ]:

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(losses)
axes[1].plot(val_rmses)
_ = axes[0].set_xlabel("Epoch")
_ = axes[1].set_xlabel("Epoch")
_ = axes[0].set_title("Training loss")
_ = axes[1].set_title("Validation RMSE")

In [ ]:
import deepsensor
from deepsensor import plot
import matplotlib.pyplot as plt

# Select a sample date for visualization
sample_date = "2012-01-01"
sample_task = task_loader(sample_date, context_sampling="all", target_sampling="all")

task_loader_periodic = TaskLoader(
    context=biased_contexts,
    target=ds_processed["TMP_2maboveground"],
)
# Plot context (biased) and target (original) data
deepsensor.plot.task(sample_task, task_loader=task_loader_periodic)

In [ ]:
# Step 1: Choose the target date for prediction
target_date = "2016-01-15"  # Example date in the 2016-2020 range

# Step 2: Load the task for the target date (context and target data)
task = task_loader(target_date, context_sampling="all", target_sampling="all")

# Step 3: Run the model to get predictions
# Note that X_t is just passed as ds. It gets the xarray structure from ds. 
pred_val = model.predict(task, X_t=ds)  # Just pass the task object

# Step 4: Extract the relevant variable for predictions (e.g., 'APCP_surface' or 'TMP_2maboveground')
# Assuming 'TMP_2maboveground' is the variable you're predicting
predxr = pred_val['TMP_2maboveground']

# Step 5: Plot the mean prediction
predxr['mean'].plot(cmap='viridis')  # Plot mean prediction
plt.title(f"Prediction for {target_date}")
plt.show()

# Step 6: Plot the standard deviation as well
predxr['std'].plot(cmap='viridis')  # Plot standard deviation
plt.title(f"Standard Deviation for {target_date}")
plt.show()

In [ ]:
pred_val